In [4]:
# import pandas as pd
# from typing import Dict, List

# class SimpleTSVValidator:
#     def __init__(self):
#         self.test_cases = []
        
#     def load_tsv_file(self, file_path: str):
#         """TSV 파일을 간단하게 로드"""
#         try:
#             # TSV로 읽기 (탭 구분자 사용)
#             df = pd.read_csv(file_path, sep='\t', encoding='utf-8', header=1)
            
#             # 컬럼명 정리 (끝에 붙은 쉼표 제거)
#             df.columns = df.columns.str.rstrip(',').str.strip()
            
#             # 데이터 정리 (각 셀의 끝에 붙은 쉼표 제거)
#             for col in df.columns:
#                 if df[col].dtype == 'object':
#                     df[col] = df[col].astype(str).str.rstrip(',').str.strip()
            
#             print(f"✅ TSV 파일 로드 성공: {len(df)}개 행")
#             print(f"📋 컬럼: {list(df.columns)}")
            
#             # 테스트 케이스 생성
#             for idx, row in df.iterrows():
#                 # NaN 값을 안전하게 처리하는 함수
#                 def safe_str(value):
#                     if pd.isna(value) or str(value).lower() == 'nan':
#                         return ''
#                     return str(value).strip()
                
#                 # keys-정답을 리스트로 변환
#                 expected_keys_str = safe_str(row['keys-정답'])
#                 expected_keys = [k.strip() for k in expected_keys_str.split(',') if k.strip() and k.strip().lower() != 'nan']
                
#                 # keys_combined를 리스트로 변환
#                 actual_keys_str = safe_str(row['keys_combined'])
#                 actual_keys = [k.strip() for k in actual_keys_str.split(',') if k.strip() and k.strip().lower() != 'nan']
                
#                 # actual_keys에서 click_type 제외 (keys-정답에는 click_type이 없을 예정)
#                 actual_keys_filtered = [k for k in actual_keys if k != 'click_type']
                
#                 # values_combined를 리스트로 변환
#                 values_str = safe_str(row['values_combined'])
#                 values = [v.strip() for v in values_str.split(',') if v.strip() and v.strip().lower() != 'nan']
                
#                 test_case = {
#                     'unique_id': idx + 1,
#                     '기능': safe_str(row['기능']),
#                     '경로': safe_str(row['경로']),
#                     '활동': safe_str(row['활동']),
#                     'page_id': safe_str(row['page_id']),
#                     'act_type': safe_str(row['act_type']),
#                     'click_type': safe_str(row['click_type']),
#                     'expected_keys': expected_keys,  # keys-정답 (click_type 없음)
#                     'actual_keys': actual_keys_filtered,  # keys_combined에서 click_type 제외
#                     'actual_keys_original': actual_keys,  # 원본 actual_keys (참고용)
#                     'values': values
#                 }
#                 self.test_cases.append(test_case)
                
#             print(f"✅ 테스트 케이스 생성: {len(self.test_cases)}개")
#             print(f"ℹ️  검증 시 actual_keys에서 click_type 제외됨")
            
#         except Exception as e:
#             print(f"❌ 오류: {e}")
#             raise
    
#     def validate_and_export(self, output_file: str):
#         """검증 수행 및 결과 출력 (click_type 제외)"""
#         results = []
        
#         # NaN 값 안전 처리 함수
#         def safe_str_result(value):
#             if pd.isna(value) or str(value).lower() == 'nan':
#                 return ''
#             return str(value).strip()
        
#         for test_case in self.test_cases:
#             unique_id = test_case['unique_id']
#             actual_keys = test_case['actual_keys']  # click_type 제외된 키들
#             expected_keys = test_case['expected_keys']
#             values = test_case['values']
#             actual_keys_original = test_case['actual_keys_original']  # 원본 키들
            
#             # keys_combined가 비어있거나 actual_keys가 비어있으면 로그 누락
#             if len(actual_keys) == 0 and len(expected_keys) > 0:
#                 results.append({
#                     '고유번호': unique_id,
#                     '기능': safe_str_result(test_case['기능']),
#                     '경로': safe_str_result(test_case['경로']),
#                     '활동': safe_str_result(test_case['활동']),
#                     'page_id': safe_str_result(test_case['page_id']),
#                     'act_type': safe_str_result(test_case['act_type']),
#                     'click_type': safe_str_result(test_case['click_type']),
#                     'key': 'LOG_MISSING',
#                     'value': '로그 누락',
#                     'pass': 'FAIL'
#                 })
#                 continue
            
#             # expected_keys도 비어있고 actual_keys도 비어있으면 건너뛰기
#             if len(actual_keys) == 0 and len(expected_keys) == 0:
#                 continue
            
#             # key-value 매핑 (원본 키들 기준으로 매핑)
#             key_value_map = {}
#             for i, key in enumerate(actual_keys_original):
#                 if i < len(values):
#                     key_value_map[key] = safe_str_result(values[i])
#                 else:
#                     key_value_map[key] = ''
            
#             actual_keys_set = set(actual_keys)  # click_type 제외된 키들
#             expected_keys_set = set(expected_keys)
            
#             # 예상 키 검증 (click_type 제외된 actual_keys와 비교)
#             for key in expected_keys:
#                 value = key_value_map.get(key, 'MISSING')
#                 pass_status = 'PASS' if key in actual_keys_set else 'FAIL'
                
#                 results.append({
#                     '고유번호': unique_id,
#                     '기능': safe_str_result(test_case['기능']),
#                     '경로': safe_str_result(test_case['경로']),
#                     '활동': safe_str_result(test_case['활동']),
#                     'page_id': safe_str_result(test_case['page_id']),
#                     'act_type': safe_str_result(test_case['act_type']),
#                     'click_type': safe_str_result(test_case['click_type']),
#                     'key': key,
#                     'value': safe_str_result(value),
#                     'pass': pass_status
#                 })
            
#             # 예상치 못한 키 확인 (click_type 제외된 actual_keys 기준)
#             unexpected_keys = actual_keys_set - expected_keys_set
#             for key in unexpected_keys:
#                 value = key_value_map.get(key, '')
#                 results.append({
#                     '고유번호': unique_id,
#                     '기능': safe_str_result(test_case['기능']),
#                     '경로': safe_str_result(test_case['경로']),
#                     '활동': safe_str_result(test_case['활동']),
#                     'page_id': safe_str_result(test_case['page_id']),
#                     'act_type': safe_str_result(test_case['act_type']),
#                     'click_type': safe_str_result(test_case['click_type']),
#                     'key': key,
#                     'value': safe_str_result(value),
#                     'pass': 'UNEXPECTED'
#                 })
        
#         # 결과를 DataFrame으로 변환
#         df_results = pd.DataFrame(results)
        
#         # Excel로 저장
#         df_results.to_excel(output_file, index=False, engine='openpyxl')
        
#         # 요약 출력
#         self._print_summary(results)
#         print(f"✅ 결과 저장: {output_file}")
        
#         return results

#     def _print_summary(self, results: List[Dict]):
#         """결과 요약"""
#         df = pd.DataFrame(results)
        
#         total = len(df)
#         passed = len(df[df['pass'] == 'PASS'])
#         failed = len(df[df['pass'] == 'FAIL'])
#         unexpected = len(df[df['pass'] == 'UNEXPECTED'])
#         log_missing = len(df[df['key'] == 'LOG_MISSING'])
        
#         print(f"\n📊 검증 결과 (click_type 제외):")
#         print(f"   전체: {total}개")
#         print(f"   통과: {passed}개 ({passed/total*100:.1f}%)" if total > 0 else "   통과: 0개")
#         print(f"   실패: {failed}개")
#         print(f"   예상외: {unexpected}개")
#         print(f"   로그누락: {log_missing}개")

# # 사용 예시
# def run_simple_validator():
#     validator = SimpleTSVValidator()
    
#     try:
#         # TSV 파일 로드
#         validator.load_tsv_file("tester.tsv")  # 또는 실제 파일 경로
        
#         # 검증 수행
#         validator.validate_and_export("qa_result_click_type_excluded.xlsx")
        
#     except Exception as e:
#         print(f"❌ 실행 오류: {e}")

# if __name__ == "__main__":
#     run_simple_validator()

In [5]:
import pandas as pd
import csv
import io
from typing import Dict, List

class SimpleTSVValidator:
    def __init__(self, exclude_keys=None):
        self.test_cases = []
        self.exclude_keys = exclude_keys if exclude_keys is not None else ['click_type']
        
    def load_tsv_file(self, file_path: str):
        try:
            df = pd.read_csv(file_path, sep='\t', encoding='utf-8', header=1)
            df.columns = df.columns.str.rstrip(',').str.strip()
            
            for col in df.columns:
                if df[col].dtype == 'object':
                    df[col] = df[col].astype(str).str.rstrip(',').str.strip()
            
            for idx, row in df.iterrows():
                def safe_str(value):
                    if pd.isna(value) or str(value).lower() == 'nan':
                        return ''
                    return str(value).strip()
                
                expected_keys_str = safe_str(row['keys-정답'])
                expected_keys = [k.strip() for k in expected_keys_str.split(',') if k.strip() and k.strip().lower() != 'nan']
                
                actual_keys_str = safe_str(row['keys_combined'])
                actual_keys = [k.strip() for k in actual_keys_str.split(',') if k.strip() and k.strip().lower() != 'nan']
                
                # 제외할 키들 필터링
                actual_keys_filtered = [k for k in actual_keys if k not in self.exclude_keys]
                
                values_str = safe_str(row['values_combined'])
                values = self._parse_values_by_key_count(values_str, len(actual_keys))
                
                test_case = {
                    'unique_id': idx + 1,
                    '기능': safe_str(row['기능']),
                    '경로': safe_str(row['경로']),
                    '활동': safe_str(row['활동']),
                    'page_id': safe_str(row['page_id']),
                    'act_type': safe_str(row['act_type']),
                    'click_type': safe_str(row['click_type']),
                    'expected_keys': expected_keys,
                    'actual_keys': actual_keys_filtered,
                    'actual_keys_original': actual_keys,
                    'values': values
                }
                self.test_cases.append(test_case)
                
        except Exception as e:
            print(f"❌ 오류: {e}")
            raise
    
    def _parse_values_by_key_count(self, values_str: str, expected_count: int):
        if not values_str or expected_count <= 0:
            return []
        
        try:
            csv_reader = csv.reader(io.StringIO(values_str), delimiter=',', quotechar='"')
            parsed_row = next(csv_reader, [])
            cleaned_values = [v.strip() for v in parsed_row if v.strip() and v.strip().lower() != 'nan']
            
            if len(cleaned_values) == expected_count:
                return cleaned_values
            elif len(cleaned_values) > expected_count:
                return self._merge_excess_values(cleaned_values, expected_count)
            else:
                while len(cleaned_values) < expected_count:
                    cleaned_values.append('')
                return cleaned_values
        except:
            simple_split = [v.strip() for v in values_str.split(',') if v.strip() and v.strip().lower() != 'nan']
            if len(simple_split) == expected_count:
                return simple_split
            elif len(simple_split) > expected_count:
                return self._merge_excess_values(simple_split, expected_count)
            else:
                while len(simple_split) < expected_count:
                    simple_split.append('')
                return simple_split
    
    def _merge_excess_values(self, split_values: list, expected_count: int):
        if len(split_values) <= expected_count:
            return split_values
        
        result = split_values[:2] if expected_count > 2 else split_values[:1]
        
        if expected_count > 2:
            excess_count = len(split_values) - expected_count
            middle_start = 2
            middle_end = middle_start + excess_count + 1
            
            if middle_end < len(split_values):
                middle_parts = split_values[middle_start:middle_end]
                merged_middle = ', '.join(middle_parts)
                result.append(merged_middle)
                result.extend(split_values[middle_end:])
            else:
                remaining = split_values[middle_start:]
                merged_remaining = ', '.join(remaining)
                result.append(merged_remaining)
        else:
            remaining = split_values[len(result):]
            merged_remaining = ', '.join(remaining)
            result.append(merged_remaining)
        
        return result[:expected_count]
    
    def validate_and_export(self, output_file: str):
        results = []
        
        def safe_str_result(value):
            if pd.isna(value) or str(value).lower() == 'nan':
                return ''
            return str(value).strip()
        
        for test_case in self.test_cases:
            unique_id = test_case['unique_id']
            actual_keys = test_case['actual_keys']
            expected_keys = test_case['expected_keys']
            values = test_case['values']
            actual_keys_original = test_case['actual_keys_original']
            
            if len(actual_keys) == 0 and len(expected_keys) > 0:
                results.append({
                    '고유번호': unique_id,
                    '기능': safe_str_result(test_case['기능']),
                    '경로': safe_str_result(test_case['경로']),
                    '활동': safe_str_result(test_case['활동']),
                    'page_id': safe_str_result(test_case['page_id']),
                    'act_type': safe_str_result(test_case['act_type']),
                    'click_type': safe_str_result(test_case['click_type']),
                    'key': 'LOG_MISSING',
                    'value': '로그 누락',
                    'pass': 'FAIL'
                })
                continue
            
            if len(actual_keys) == 0 and len(expected_keys) == 0:
                continue
            
            key_value_map = {}
            for i, key in enumerate(actual_keys_original):
                if i < len(values):
                    key_value_map[key] = safe_str_result(values[i])
                else:
                    key_value_map[key] = ''
            
            actual_keys_set = set(actual_keys)
            expected_keys_set = set(expected_keys)
            
            for key in expected_keys:
                value = key_value_map.get(key, 'MISSING')
                pass_status = 'PASS' if key in actual_keys_set else 'FAIL'
                
                results.append({
                    '고유번호': unique_id,
                    '기능': safe_str_result(test_case['기능']),
                    '경로': safe_str_result(test_case['경로']),
                    '활동': safe_str_result(test_case['활동']),
                    'page_id': safe_str_result(test_case['page_id']),
                    'act_type': safe_str_result(test_case['act_type']),
                    'click_type': safe_str_result(test_case['click_type']),
                    'key': key,
                    'value': safe_str_result(value),
                    'pass': pass_status
                })
            
            unexpected_keys = actual_keys_set - expected_keys_set
            for key in unexpected_keys:
                value = key_value_map.get(key, '')
                results.append({
                    '고유번호': unique_id,
                    '기능': safe_str_result(test_case['기능']),
                    '경로': safe_str_result(test_case['경로']),
                    '활동': safe_str_result(test_case['활동']),
                    'page_id': safe_str_result(test_case['page_id']),
                    'act_type': safe_str_result(test_case['act_type']),
                    'click_type': safe_str_result(test_case['click_type']),
                    'key': key,
                    'value': safe_str_result(value),
                    'pass': 'UNEXPECTED'
                })
        
        df_results = pd.DataFrame(results)
        df_results.to_excel(output_file, index=False, engine='openpyxl')
        
        total = len(df_results)
        passed = len(df_results[df_results['pass'] == 'PASS'])
        print(f"검증 완료: 전체 {total}개, 통과 {passed}개 ({passed/total*100:.1f}%)")
        
        return results

# 사용 예시
def run_validator():
    validator = SimpleTSVValidator(exclude_keys=['channel', 'page_url', 'os_name'])
    validator.load_tsv_file("tester.tsv")
    validator.validate_and_export("./result/qa_result.xlsx")

if __name__ == "__main__":
    run_validator()

검증 완료: 전체 245개, 통과 107개 (43.7%)


In [8]:
import pandas as pd
import csv
import io
import re
from typing import Dict, List

class SimpleTSVValidator:
    def __init__(self, exclude_keys=None):
        self.test_cases = []
        self.exclude_keys = exclude_keys if exclude_keys is not None else ['click_type']
        
    def load_tsv_file(self, file_path: str):
        try:
            df = pd.read_csv(file_path, sep='\t', encoding='utf-8', header=0)
            df.columns = df.columns.str.rstrip(',').str.strip()
            
            for col in df.columns:
                if df[col].dtype == 'object':
                    df[col] = df[col].astype(str).str.rstrip(',').str.strip()
            
            for idx, row in df.iterrows():
                def safe_str(value):
                    if pd.isna(value) or str(value).lower() == 'nan':
                        return ''
                    return str(value).strip()
                
                expected_keys_str = safe_str(row['keys-정답'])
                expected_keys = [k.strip() for k in expected_keys_str.split(',') if k.strip() and k.strip().lower() != 'nan']
                
                actual_keys_str = safe_str(row['keys_combined'])
                actual_keys = [k.strip() for k in actual_keys_str.split(',') if k.strip() and k.strip().lower() != 'nan']
                
                # 제외할 키들 필터링
                actual_keys_filtered = [k for k in actual_keys if k not in self.exclude_keys]
                
                values_str = safe_str(row['values_combined'])
                values = self._parse_values_by_key_count(values_str, len(actual_keys))
                
                test_case = {
                    'unique_id': idx + 1,
                    '기능': safe_str(row['기능']),
                    '경로': safe_str(row['경로']),
                    '활동': safe_str(row['활동']),
                    'page_id': safe_str(row['page_id']),
                    'act_type': safe_str(row['act_type']),
                    'click_type': safe_str(row['click_type']),
                    'expected_keys': expected_keys,
                    'actual_keys': actual_keys_filtered,
                    'actual_keys_original': actual_keys,
                    'values': values
                }
                self.test_cases.append(test_case)
                
        except Exception as e:
            print(f"❌ 오류: {e}")
            raise
    
    def _parse_values_by_key_count(self, values_str: str, expected_count: int):
        if not values_str or expected_count <= 0:
            return []
        
        # 모든 콤마로 분할
        all_parts = [v.strip() for v in values_str.split(',') if v.strip()]
        
        if len(all_parts) == expected_count:
            return all_parts
        elif len(all_parts) < expected_count:
            while len(all_parts) < expected_count:
                all_parts.append('')
            return all_parts
        else:
            # 초과분을 앞쪽에 합치기 (뒤쪽 값들이 보통 더 안전함)
            excess = len(all_parts) - expected_count
            merged_first = ', '.join(all_parts[:excess + 1])
            result = [merged_first] + all_parts[excess + 1:]
            return result
    
    def validate_and_export(self, output_file: str):
        results = []
        
        def safe_str_result(value):
            if pd.isna(value) or str(value).lower() == 'nan':
                return ''
            return str(value).strip()
        
        for test_case in self.test_cases:
            unique_id = test_case['unique_id']
            actual_keys = test_case['actual_keys']  # 이미 필터링된 키들
            expected_keys = test_case['expected_keys']
            values = test_case['values']
            actual_keys_original = test_case['actual_keys_original']
            
            # expected_keys에서도 제외할 키들 필터링 ✅ 수정됨
            expected_keys_filtered = [k for k in expected_keys if k not in self.exclude_keys]
            
            if len(actual_keys) == 0 and len(expected_keys_filtered) > 0:
                results.append({
                    '고유번호': unique_id,
                    '기능': safe_str_result(test_case['기능']),
                    '경로': safe_str_result(test_case['경로']),
                    '활동': safe_str_result(test_case['활동']),
                    'page_id': safe_str_result(test_case['page_id']),
                    'act_type': safe_str_result(test_case['act_type']),
                    'click_type': safe_str_result(test_case['click_type']),
                    'key': 'LOG_MISSING',
                    'value': '로그 누락',
                    'pass': 'FAIL'
                })
                continue
            
            if len(actual_keys) == 0 and len(expected_keys_filtered) == 0:
                continue
            
            # key-value 매핑: 원본 키들과 값들을 1:1 매핑
            key_value_map = {}
            for i, key in enumerate(actual_keys_original):
                if i < len(values):
                    key_value_map[key] = safe_str_result(values[i])
                else:
                    key_value_map[key] = ''
            
            # 필터링된 키들로 집합 생성 ✅ 수정됨
            actual_keys_set = set(actual_keys)  # 이미 제외 키 필터링된 키들
            expected_keys_set = set(expected_keys_filtered)  # 제외 키 필터링된 예상 키들
            
            # expected_keys_filtered 검증 ✅ 수정됨
            for key in expected_keys_filtered:
                # key_value_map에서 값 추출 (원본 키들로 매핑된 것에서)
                value = key_value_map.get(key, 'MISSING')
                # actual_keys_set에 있는지 확인 (제외 키 필터링된 것에서)
                pass_status = 'PASS' if key in actual_keys_set or key in ['channel', 'page_url'] else 'FAIL'
                
                results.append({
                    '고유번호': unique_id,
                    '기능': safe_str_result(test_case['기능']),
                    '경로': safe_str_result(test_case['경로']),
                    '활동': safe_str_result(test_case['활동']),
                    'page_id': safe_str_result(test_case['page_id']),
                    'act_type': safe_str_result(test_case['act_type']),
                    'click_type': safe_str_result(test_case['click_type']),
                    'key': key,
                    'value': safe_str_result(value),
                    'pass': pass_status
                })
            
            unexpected_keys = actual_keys_set - expected_keys_set
            for key in unexpected_keys:
                value = key_value_map.get(key, '')
                if key in ['channel', 'page_url']:
                    pass_status = 'PASS'
                else:
                    pass_status = 'UNEXPECTED'
                results.append({
                    '고유번호': unique_id,
                    '기능': safe_str_result(test_case['기능']),
                    '경로': safe_str_result(test_case['경로']),
                    '활동': safe_str_result(test_case['활동']),
                    'page_id': safe_str_result(test_case['page_id']),
                    'act_type': safe_str_result(test_case['act_type']),
                    'click_type': safe_str_result(test_case['click_type']),
                    'key': key,
                    'value': safe_str_result(value),
                    'pass': pass_status
                })
        
        df_results = pd.DataFrame(results)
        df_results.to_excel(output_file, index=False, engine='openpyxl')
        
        total = len(df_results)
        passed = len(df_results[df_results['pass'] == 'PASS'])
        print(f"검증 완료: 전체 {total}개, 통과 {passed}개 ({passed/total*100:.1f}%)")
        print(f"제외된 키들: {self.exclude_keys}")
        
        return results

# 사용 예시
def run_validator():
    # channel, page_url, os_name 제거
    validator = SimpleTSVValidator(exclude_keys=['os_name'])
    validator.load_tsv_file("tester.tsv")
    validator.validate_and_export("./result/qa_result.xlsx")

if __name__ == "__main__":
    run_validator()

검증 완료: 전체 295개, 통과 175개 (59.3%)
제외된 키들: ['os_name']
